# 例子

造房子，有建造者和指挥者，建造者执行具体任务，指挥者知道每个步骤都干什么。

In [1]:
class Builder:
    
    def get_material(self):
        print("搬运材料中...")
        return self
    
    def build(self):
        print("建造中...")
        return self
    
    def complete(self):
        print("交付...")
        return "房子"

In [2]:
# 直接指挥 Builder 去干活

Builder().get_material().build().complete()

搬运材料中...
建造中...
交付...


'房子'

In [3]:
class Director:
    
    def __init__(
        self,
        builder,
    ) -> None:
        self.builder = builder

    def direct(self):
        building = self.builder.get_material().build().complete()
        print(f"{building} 已经建造完成")
        return building

In [4]:
# 另外找一个已经知道怎么指挥人干活的人指挥人来干活
builder = Builder()
director = Director(builder)
building = director.direct()
building

搬运材料中...
建造中...
交付...
房子 已经建造完成


'房子'

## 另一个例子：装机

In [5]:
class PC:

    def __init__(
        self
    ) -> None:
        self.cpu = None
        self.mem = None
        self.gpu = None
        self.storage = None

    # def __str__(self) -> str:
    def 看看鲁大师(self) -> str:
        return f"CPU: {self.cpu} Mem: {self.mem} GPU: {self.gpu} Storage: {self.storage}"

In [6]:
class PCBuilder:

    def __init__(self) -> None:
        self.pc = PC()
        
    def add_cpu(self, cpu) -> None:
        self.pc.cpu = cpu

    def add_mem(self, mem) -> None:
        self.pc.mem = mem

    def add_gpu(self, gpu) -> None:
        self.pc.gpu = gpu

    def add_storage(self, storage) -> None:
        self.pc.storage = storage

    def get_pc(self) -> PC:
        return self.pc

In [7]:
class 装机员:

    def __init__(
        self,
        builder,
    ) -> None:
        self.builder = builder

    def 装机(self, cpu, mem, gpu, storage) -> PC:
        self.builder.add_cpu(cpu)
        self.builder.add_mem(mem)
        self.builder.add_gpu(gpu)
        self.builder.add_storage(storage)
        pc = self.builder.get_pc()
        return pc

In [8]:
new_pc = 装机员(PCBuilder()).装机(cpu="i9-13900K", mem="64GB", gpu="RTX4090", storage="1T")
new_pc.看看鲁大师()

'CPU: i9-13900K Mem: 64GB GPU: RTX4090 Storage: 1T'

## 思考

- 这个和上面的有什么不同？
- Navigator 的开发与使用过程中可以在哪些地方用到？

- 复杂 `generator` 与 `optimizer` 的内部，涉及多个模块，按需构建出不同功能的 module
- dataflow, `generator` → `optimizer` → `scoring`
- 对于一些比较固定 or 给出推荐可选搭配的 workflow，作为上层封装使用

## 和工厂模式的区别

和工厂模式不一样，工厂模式是直接返回一个产品（或工厂），输入输出，相当直接，建造者模式就要辛苦一点，需要 **指挥者** 指挥 **建造者** 工作之后，才能获得产品，所以建造者模式需要 **指挥者** 和 **建造者** 两个角色。

## 适用场景（相对于工厂模式）

- 想要创建一个复杂对象（对象由多个部分构成，且对象的创建要经过多个不同的步骤，这些步骤也许还需遵从特定的顺序）
- 要求一个对象能有不同的表现，并希望将对象的构造与表现解耦
- 想要在某个时间点创建对象，但在稍后的时间点再访问

# 其他的一些 Tips

第一个例子里有这样的调用方式

```python3
Builder().get_material().build().complete()
```

为什么可以这样做呢？因为 `Builder()`，`get_material()` 和 `build()` 都返回了 `Builder` 对象，后面的一些方法都是 `Builder` 这个 class 定义好的，所以这个对象可以依次 `.func()` 过去。

这个东西叫 **方法链式调用 (Method chaining)**

是面向对象编程语言中多个方法被调用时的常用语法。每个方法都返回一个对象，允许在单个语句中将调用链接在一起，而无需变量来存储中间结果。

对比第二个例子中的 `PCBuilder` 类，内部在用 `self.pc` 存储 `PC` 对象

一方面链式调用确实写起来优雅一些，但对于 debug 其实是更困难的，同时 Python 写链式调用其实有点别扭，在这个例子中能随便 `.fun()` 过去是因为这些 `.fun()` 都是绑定在对象上的。

Python 还有一些高阶函数，如 `map`, `filter`, `reduce`，但个人感觉用起来很别扭。

In [9]:
arr = [1, 2, 3, 4, 5]
res = map(lambda x: x**2, arr)
res = filter(lambda x: x > 10, res)
res = list(res) # 迭代器 -> 列表
res

[16, 25]

或者你也不想写出这样的代码被人追杀吧

In [10]:
arr = [1, 2, 3, 4, 5]
res = list(filter(lambda x: x > 10, map(lambda x: x**2, arr)))
res

[16, 25]

如果对函数式语言或者比较函的语言（比如 Julia），可以写成这样

```julia
julia> [1, 2, 3, 4, 5] |> x -> x.^2 |> filter(x -> x > 10)
3-element Vector{Int64}:
  9
 16
 25
```

其中 `x |> f` 的调用方式等价于 `f(x)`，而在这里 `x -> x.^2` 是个匿名 Lambda 函数

当然这种已经是语言设计级别上的语法糖了（逃）